In [ ]:
import os
import pandas as pd
from shutil import copyfile,rmtree
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# getting the list of all train images
data = pd.read_csv("/Downloads/state-farm-distracted-driver-detection/driver_imgs_list.csv", usecols = [1,2])

# list of all classes
classes_list = data['classname'].unique()

# dictionary containing all train data file names, class wise
train_data_files={}
for cls, image_name in data.values:
    key = cls
    if key in train_data_files:
        train_data_files[key].append(image_name)
    else:
        train_data_files[key] = [image_name]
        
BATCH_SIZE = 128
IMAGE_SIZE = 224

TRAIN_DIR='/Downloads/state-farm-distracted-driver-detection/imgs/train'
TEST_DIR='/Downloads/state-farm-distracted-driver-detection/imgs/test'
datagen = ImageDataGenerator(
        rescale = 1./255,
        validation_split = 0.2
)

training_data = datagen.flow_from_directory(TRAIN_DIR,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=BATCH_SIZE,
                                        subset='training',shuffle=False)

testing_data = datagen.flow_from_directory(TEST_DIR,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=BATCH_SIZE,
                                        subset='testing',shuffle=False)

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
model = keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(10, activation='softmax')
])

# compile the model
model.compile(optimizer= Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

MIN_DELTA=0.005
EPOCHS=20
PATIENCE=2
# to stop training if no significant change in validation data accuracy
es = EarlyStopping(monitor = 'val_accuracy', patience = PATIENCE, min_delta = MIN_DELTA)

# fitting and generating the model
model.fit(
    training_data, 
    epochs = EPOCHS, 
    validation_data = testing_data,
    callbacks = [es]
         )
from tensorflow.keras.utils import image_dataset_from_directory
img_dir="/Downloads/state-farm-distracted-driver-detection/imgs"
# transforming test data same as training data
test_data_gen = ImageDataGenerator(
    rescale = 1./225
)
test_data = test_data_gen.flow_from_directory(img_dir,
                          target_size = (IMAGE_SIZE, IMAGE_SIZE),
                          classes = ['test'],
                          shuffle = False,
                          batch_size = BATCH_SIZE)
predicted = model.predict(test_data)
test_data_files = image_dataset_from_directory(
    '/Downloads/state-farm-distracted-driver-detection/imgs/test',
     labels = None,
    label_mode=None,
)
df = pd.DataFrame(predicted)
df.columns = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
filepath = [i.split('/')[-1] for i in test_data_files.file_paths]
df1 = pd.DataFrame(filepath)
df1.columns = ['img']
df = df1.join(df)
df.to_csv('/kaggle/working/output.csv',index=False)
